# import

In [1]:
import sys
import requests
import tarfile
import json
import numpy as np
from os import path
from PIL import Image
from PIL import ImageFont, ImageDraw
from glob import glob
from matplotlib import pyplot as plt
import cv2

import random
from tqdm import tqdm

import re
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import CSVLogger

from keras.applications import MobileNetV2

from keras.layers import *
from keras.models import Model

import zipfile

Using TensorFlow backend.


In [2]:
def WriteToFile(Name, InhaltInArrayForm):
  file = open(Name, 'w')
  for i in InhaltInArrayForm:
    file.write(i)
    file.write("\n")
  file.close()

# laden der Daten

## Download

In [3]:
import zipfile
def unpackZip(path_to_zip_file, directory_to_extract_to):
  with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
      zip_ref.extractall(directory_to_extract_to)
  print("DONE")

In [4]:
# DatenLaden
!curl http://images.cocodataset.org/zips/train2014.zip --output "train2014.zip"
unpackZip("train2014.zip", "content/")
!rm train2014.zip

#!curl http://images.cocodataset.org/zips/val2014.zip --output "val2017.zip"
#!unzip val2017.zip
#!rm val2017.zip

#!curl http://images.cocodataset.org/zips/test2014.zip --output "test2017.zip"
#!unzip test2017.zip
#!rm test2017.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12.5G  100 12.5G    0     0  34.8M      0  0:06:09  0:06:09 --:--:-- 33.7M
DONE


## Umwandeln in "Nützliche Daten"

In [5]:
StringToJson = "drive/My Drive/data/cocotext.v2.json"
DataPath = "train2014/"

In [6]:
ZielSpeicher = []

### Aufbereitung

In [7]:
with open(StringToJson, 'r') as fp:
    samples = json.load(fp)

anns = samples['anns']
imgs = samples['imgs']

BauArray = {}


In [8]:
tempSave = {}

for i, (_, image) in tqdm(enumerate(imgs.items())):
  temp = {}
  imageID = image['id']
  temp['file_name'] = image['file_name']
  temp['set'] = image['set']
  temp['boxes'] = []
  
  tempSave[image['set']] = 1

  BauArray[str(imageID)] = temp

53686it [00:00, 571014.08it/s]


In [9]:
for i, (_, image) in tqdm(enumerate(anns.items())):
  img_ID = image['image_id']
  img_Box = image['bbox']

  
  BauArray[str(img_ID)]['boxes'].append(img_Box)

201126it [00:00, 605939.11it/s]


In [10]:
for i, (_, image) in tqdm(enumerate(BauArray.items())):
  WorkingObject = image
  if len(WorkingObject['boxes']) != 0:
    ZielSpeicher.append(WorkingObject)
  

53686it [00:00, 1564652.53it/s]


# Darknet

In [11]:
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

Cloning into 'darknet'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 13892 (delta 3), reused 6 (delta 2), pack-reused 13877
Receiving objects: 100% (13892/13892), 12.44 MiB | 16.04 MiB/s, done.
Resolving deltas: 100% (9485/9485), done.
/content/darknet


In [12]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

In [13]:
cd ../

/content


# Transformieren

In [14]:
def convert_labels(path, b0, b1, b2, b3):

    #[x,y,width,height]

    x = b0
    y = b1
    w = b2
    h = b3

    size = get_img_shape(path)
    dw = 1./size[1]
    dh = 1./size[0]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return x,y,w,h


def get_img_shape(path):
    path = path
    img = cv2.imread(path)
    try:
        return img.shape
    except AttributeError:
        #print('error! ', path)
        return (None, None, None)

In [25]:
def CreateFiles(Bild):
  NameDesBildes = Bild['file_name']
  Boxes = Bild['boxes']
  SetName = Bild['set']

  StringZumSpeichern = "../content/train2014/"

  LinkToFile = StringZumSpeichern + NameDesBildes.split('.')[0] + '.txt'

  Inhalt = []
  
  for Box in Boxes:
    Category = 0
    #[x,y,width,height]
    x,w,y,h = convert_labels("../content/train2014/" + NameDesBildes, Box[0], Box[1], Box[2], Box[3])

    TempString = str(Category)
    TempString += " " + str(x)
    TempString += " " + str(w)
    TempString += " " + str(y)
    TempString += " " + str(h)

    Inhalt.append(TempString)
    # print(FileName + "     " + TempString)
    
  
  # Image = cv2.imread("../train2014/train2014/" + NameDesBildes)
  # cv2.imwrite(StringZumSpeichern + NameDesBildes, Image)

  InhaltFürTrain.append(StringZumSpeichern + NameDesBildes)
  WriteToFile(LinkToFile, Inhalt)
  return True

In [23]:
!ls
%cd darknet

3rdparty   CMakeLists.txt	   image_yolov3.sh	  README.md
backup	   darknet		   include		  results
build	   DarknetConfig.cmake.in  json_mjpeg_streams.sh  scripts
build.ps1  darknet.py		   LICENSE		  src
build.sh   darknet_video.py	   Makefile		  video_v2.sh
cfg	   data			   net_cam_v3.sh	  video_yolov3.sh
cmake	   image_yolov2.sh	   obj
[Errno 20] Not a directory: 'darknet'
/content/darknet


In [26]:
!mkdir TrainData
%cd darknet/
# --------------

InhaltFürTrain = []
PathToTrain = 'data/train.txt'

counterFalsch = 0
counterCorrect = 0

for Bild in tqdm(ZielSpeicher):
  try:
    CreateFiles(Bild)
    counterCorrect += 1
  except Exception:
    counterFalsch += 1

WriteToFile(PathToTrain, InhaltFürTrain)
print("\n" + "\n" + "Korrekt erfasst..................... " + str(counterCorrect))
print("Fehler..............................." + str(counterFalsch))

mkdir: cannot create directory ‘TrainData’: File exists


  0%|          | 1/23485 [00:00<41:01,  9.54it/s]

[Errno 20] Not a directory: 'darknet/'
/content/darknet


100%|██████████| 23485/23485 [19:59<00:00, 19.59it/s]



Korrekt erfasst..................... 23485
Fehler...............................0


## Obj Names


In [18]:
Name = 'obj.names'
Zusatz = 'data/'
FileName = Zusatz + Name
Inhalt = []

Inhalt.append("Text")

WriteToFile(FileName, Inhalt)

## Obj Data

In [19]:
!mkdir ../backup

Inhalt = [
          'classes = 1',
          'train = data/train.txt',
          'names = data/obj.names',
          'backup = ../backup'
]
PathToObjData = 'data/obj.data'

WriteToFile(PathToObjData, Inhalt)

# Training




In [29]:
%cd darknet

# ! ./darknet detector train data/obj.data ../drive/My\ Drive/data/yolov3_custom.cfg ../drive/My\ Drive/data/darknet53.conv.74 -dont_show

# Falls es zwischendurch abstürzt:
! ./darknet detector train data/obj.data ../drive/My\ Drive/data/yolov3_custom.cfg ../drive/My\ Drive/backup/yolov3_custom_5000.weights -dont_show

Streaming output truncated to the last 5000 lines.
 9926: 2.987227, 3.366504 avg loss, 0.000010 rate, 5.767694 seconds, 635264 images, 0.174444 hours left
Loaded: 0.000045 seconds
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000089, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.002091, iou_loss = 0.000000, total_loss = 0.002091 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.727695, GIOU: 0.713147), Class: 0.998946, Obj: 0.228251, No Obj: 0.000226, .5R: 1.000000, .75R: 0.400000, count: 5, class_loss = 0.813176, iou_loss = 0.439841, total_loss = 1.253017 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.554647, GIOU: 0.541249), Class: 0.998808, Obj: 0.209575, No Obj: 0.000436, .5R: 0.529412, .75R: 0.294118, count: 17, class_loss = 3.761105, iou_loss = 3.498650, total_loss = 7.259756 
 total_bbox = 2194975, rewritten_bbox = 3.736352 % 
v3 (

# Test

In [30]:
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov3_custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov3_custom.cfg
!%cd ..

/content/darknet/cfg
sed: can't read yolov3_custom.cfg: No such file or directory
sed: can't read yolov3_custom.cfg: No such file or directory
/content/darknet


In [5]:
%cd darknet

!./darknet detector test data/obj.data ../drive/My\ Drive/data/yolov3_vorTest.cfg ../drive/My\ Drive/backup/yolov3_custom_final.weights ../drive/My\ Drive/data/Test.png -thresh 0.4

Ergebnis = Image.open('predictions.jpg')

plt.figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')

plt.imshow(Ergebnis)

[Errno 20] Not a directory: 'darknet'
/content/darknet
 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 S

NameError: ignored